This notebook generate text using a Markov chain based on the text from the chat messages

In [1]:
import psycopg2
conn = psycopg2.connect(user="postgres", password="mysecretpassword", host="localhost")
# should be UTF-8
print('connection encoding, used as default:', conn.encoding)

connection encoding, used as default: UTF8


Now use the DB to generate sequences

In [3]:
import time
import random

# stopwords_it = set(["a","ad","agl","agli","ai","al","alcuna","alcuni","alcuno","all","alla","alle","allo","allora","altri","altro","anche","ancora","aver","avere","averlo","avesse","avessero","avessi","avessimo","aveste","avesti","avete","aveva","avevamo","avevano","avevate","avevi","avevo","avrai","avranno","avrebbe","avrebbero","avrei","avremmo","avremo","avreste","avresti","avrete","avrà","avrò","avuta","avute","avuti","avuto",
#                    "basta","è","il","le", "che", "un","una","uno","perché","mi","di","ma","per","in","non","e","si","sto","la","lo","gli","con","nel","già","da","i","sono","come","se","io","ho","più","però","ti","su","hai","fa","per","tra","fra","me","ha","ci","del","poi","o","cosa","cose","mai","fare"])

external_token = '**EXTERNAL**'

def continuations(first_token, num=5):
    # table bigrams: (token1, token2, counter)
    named_cur = conn.cursor('window_read_cursor')
    named_cur.execute("SELECT SUM(counter) as total FROM bigrams WHERE token1 = %s", (first_token,))
    total = named_cur.fetchone()[0]
    named_cur.close()
    
    chosen_totals = [random.randint(0, total) for i in range(num)]
    chosen_tokens = []
    # print(f'there are {total} know successors to {first_token}, chosen ones are #{chosen_totals}')
    
    named_cur = conn.cursor('window_read_cursor')
    named_cur.execute("SELECT token2, counter FROM bigrams WHERE token1 = %s", (first_token,))
    for record in named_cur:
        subtraction_value = int(record[1])
        chosen_totals = [c - subtraction_value for c in chosen_totals]
        for ct in chosen_totals:
            if ct < 0 and ct + subtraction_value > 0:
                #print(f'adding successor {record[0]}')
                chosen_tokens.append(record[0])
                
        if all(map(lambda k: k < 0, chosen_totals)):
            break
    named_cur.close()
    return sorted(chosen_tokens)

sample_question = 'Milano non è poi così brutta, no?'.lower().split()

current_token = None
chosen_answer = []
while current_token != '*':
    if current_token is None:
        current_token = '*'
    next_candidates = continuations(current_token, num=400)
    named_cur = conn.cursor('window_read_cursor')
    named_cur.execute("select word_a, SUM(relevance) from relevances where word_q = ANY(%s) AND word_q = ANY(%s)  GROUP BY word_a;", (sample_question, next_candidates))
    # counter for how many times this word was used replying to messges containing the question words
    qa_counts = [0.00000000000000001] * len(next_candidates)

    for record in named_cur:
        #print('raw record', record)
        if record[0] in next_candidates:
            qa_counts[next_candidates.index(record[0])] = float(record[1])

    named_cur.close()
    # print('qa_counts:', qa_counts)
    # now pick a continuation based on the co-occurrency counts
    chosen_sum = random.random() * sum(qa_counts)
    for i in range(len(qa_counts)):
        chosen_sum -= qa_counts[i]
        if chosen_sum <= 0:
            current_token = next_candidates[i]
            print('chose', current_token)
            print('partial answer:', ' '.join(chosen_answer))
            chosen_answer.append(next_candidates[i])
            break
    # randomly end
print('final result', chosen_answer)

chose poi
partial answer: 
chose che
partial answer: poi
chose ne
partial answer: poi che
chose faccio
partial answer: poi che ne
chose da
partial answer: poi che ne faccio
chose che
partial answer: poi che ne faccio da
chose fare
partial answer: poi che ne faccio da che
chose che
partial answer: poi che ne faccio da che fare
chose sì
partial answer: poi che ne faccio da che fare che
chose non
partial answer: poi che ne faccio da che fare che sì
chose solo
partial answer: poi che ne faccio da che fare che sì non
chose con
partial answer: poi che ne faccio da che fare che sì non solo
chose l'altro
partial answer: poi che ne faccio da che fare che sì non solo con
chose che
partial answer: poi che ne faccio da che fare che sì non solo con l'altro
chose a
partial answer: poi che ne faccio da che fare che sì non solo con l'altro che
chose una
partial answer: poi che ne faccio da che fare che sì non solo con l'altro che a
chose volta
partial answer: poi che ne faccio da che fare che sì non s